### Building Parameters

The building has a **stepped structure** with centered setback:

| Component | Width (X) | Depth (Y) | Floors | Height |
|-----------|-----------|-----------|--------|--------|
| **Podium (Base)** | 5 bays × 8m = 40m | 2 bays × 8m = 16m | Floors 0-12 (13 floors) | 81m |
| **Tower** | 3 bays × 8m = 24m | 2 bays × 8m = 16m | Floors 13-25 (13 floors) | 159m |

The tower is **centered** - one column removed from each side (left and right).

| Parameter | Value |
|-----------|-------|
| **Ground Floor Height** | 9.0m |
| **Typical Floor Height** | 6.0m |
| **Total Floors** | 26 (Floor 0 to Floor 25) |
| **Total Height** | 9 + 24×6 = 153m |

---
## 1. Import Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from itertools import combinations
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

---
## 2. Define Building Geometry Parameters

**Column Grid:**
- Podium X: 0, 8, 16, 24, 32, 40 (6 column lines)
- Tower X: 8, 16, 24, 32 (4 column lines - centered, skip edges)
- Y: 0, 8, 16 (3 column lines - same for both)

In [2]:
# ============================================
# BUILDING GEOMETRY PARAMETERS
# ============================================

# Bay dimensions
BAY_WIDTH = 8.0  # meters (same in X and Y)

# PODIUM (Base) - Full footprint, floors 0-12 (13 floors)
PODIUM_BAYS_X = 5  # 5 bays = 40m width
PODIUM_BAYS_Y = 2  # 2 bays = 16m depth
PODIUM_FLOORS = 13  # Floors 0 to 12 (13 floors)

# TOWER - Centered reduced footprint, floors 13-25 (13 floors)
TOWER_BAYS_X = 3   # 3 bays = 24m width (centered - remove 1 column each side)
TOWER_BAYS_Y = 2   # 2 bays = 16m depth (full depth)
TOTAL_FLOORS = 26  # Floors 0 to 25 (26 floors total, ground floor included)

# Floor heights
GROUND_FLOOR_HEIGHT = 9.0  # meters
TYPICAL_FLOOR_HEIGHT = 6.0  # meters

# Grid coordinates
# Podium X: 0, 8, 16, 24, 32, 40 (6 column lines)
PODIUM_X_COORDS = np.array([i * BAY_WIDTH for i in range(PODIUM_BAYS_X + 1)])

# Tower X: 8, 16, 24, 32 (4 column lines - centered, skip first and last)
TOWER_X_COORDS = np.array([8.0, 16.0, 24.0, 32.0])  # Centered columns

# Y: 0, 8, 16 (3 column lines) - same for both
Y_COORDS = np.array([i * BAY_WIDTH for i in range(PODIUM_BAYS_Y + 1)])

# Calculate Z coordinates for each floor
Z_COORDS = [0.0]  # Floor 0 at z=0
Z_COORDS.append(GROUND_FLOOR_HEIGHT)  # Floor 1 at z=9
for i in range(2, TOTAL_FLOORS):
    Z_COORDS.append(Z_COORDS[-1] + TYPICAL_FLOOR_HEIGHT)
Z_COORDS = np.array(Z_COORDS)

---
## 3. Generate Position Matrix (Node Coordinates)

For a stepped building with centered setback:
- **Podium floors (0-12)**: Full 6×3 grid = 18 nodes per floor
- **Tower floors (13-25)**: Centered 4×3 grid = 12 nodes per floor

In [3]:
def generate_stepped_position_matrix():
    """
    Generate position matrix for stepped building (podium + centered tower).
    
    Returns:
        DataFrame with columns: node_id, x, y, z, floor, grid_x, grid_y, zone
    """
    nodes = []
    node_id = 0
    
    for floor_idx, z in enumerate(Z_COORDS):
        if floor_idx < PODIUM_FLOORS:
            # Podium floors - full footprint
            x_coords = PODIUM_X_COORDS
            zone = 'podium'
        else:
            # Tower floors - centered reduced footprint
            x_coords = TOWER_X_COORDS
            zone = 'tower'
        
        for y_idx, y in enumerate(Y_COORDS):
            for x_idx, x in enumerate(x_coords):
                nodes.append({
                    'node_id': node_id,
                    'x': x,
                    'y': y,
                    'z': z,
                    'floor': floor_idx,
                    'grid_x': x_idx,
                    'grid_y': y_idx,
                    'zone': zone,
                    'label': f"N{node_id}_F{floor_idx}"
                })
                node_id += 1
    
    return pd.DataFrame(nodes)

# Generate position matrix
position_df = generate_stepped_position_matrix()

# Calculate expected node counts
podium_nodes_per_floor = len(PODIUM_X_COORDS) * len(Y_COORDS)  # 6 × 3 = 18
tower_nodes_per_floor = len(TOWER_X_COORDS) * len(Y_COORDS)    # 4 × 3 = 12
total_podium_nodes = podium_nodes_per_floor * PODIUM_FLOORS
total_tower_nodes = tower_nodes_per_floor * (TOTAL_FLOORS - PODIUM_FLOORS)

print("=" * 70)
print("POSITION MATRIX")
print("=" * 70)
print(f"\n🔢 NODE COUNTS:")
print(f"   Podium floors (0-{PODIUM_FLOORS-1}): {podium_nodes_per_floor} nodes/floor × {PODIUM_FLOORS} floors = {total_podium_nodes} nodes")
print(f"   Tower floors ({PODIUM_FLOORS}-{TOTAL_FLOORS-1}): {tower_nodes_per_floor} nodes/floor × {TOTAL_FLOORS - PODIUM_FLOORS} floors = {total_tower_nodes} nodes")
print(f"   TOTAL: {total_podium_nodes + total_tower_nodes} nodes")
print(f"\n   Actual count: {len(position_df)} nodes")

print(f"\n📊 Position Matrix Shape: {position_df.shape}")

POSITION MATRIX

🔢 NODE COUNTS:
   Podium floors (0-12): 18 nodes/floor × 13 floors = 234 nodes
   Tower floors (13-25): 12 nodes/floor × 13 floors = 156 nodes
   TOTAL: 390 nodes

   Actual count: 390 nodes

📊 Position Matrix Shape: (390, 9)


In [4]:
# Display nodes by zone
print("\n=== PODIUM NODES (Floor 0) ===")
display(position_df[(position_df['floor'] == 0)])

print("\n=== TOWER NODES (Floor 10 - first tower floor) ===")
display(position_df[(position_df['floor'] == 10)])


=== PODIUM NODES (Floor 0) ===


,node_id,x,y,z,floor,grid_x,grid_y,zone,label
0,0,0.0,0.0,0.0,0,0,0,podium,N0_F0
1,1,8.0,0.0,0.0,0,1,0,podium,N1_F0
2,2,16.0,0.0,0.0,0,2,0,podium,N2_F0
3,3,24.0,0.0,0.0,0,3,0,podium,N3_F0
4,4,32.0,0.0,0.0,0,4,0,podium,N4_F0
5,5,40.0,0.0,0.0,0,5,0,podium,N5_F0
6,6,0.0,8.0,0.0,0,0,1,podium,N6_F0
7,7,8.0,8.0,0.0,0,1,1,podium,N7_F0
8,8,16.0,8.0,0.0,0,2,1,podium,N8_F0
9,9,24.0,8.0,0.0,0,3,1,podium,N9_F0



=== TOWER NODES (Floor 10 - first tower floor) ===


,node_id,x,y,z,floor,grid_x,grid_y,zone,label
180,180,0.0,0.0,63.0,10,0,0,podium,N180_F10
181,181,8.0,0.0,63.0,10,1,0,podium,N181_F10
182,182,16.0,0.0,63.0,10,2,0,podium,N182_F10
183,183,24.0,0.0,63.0,10,3,0,podium,N183_F10
184,184,32.0,0.0,63.0,10,4,0,podium,N184_F10
185,185,40.0,0.0,63.0,10,5,0,podium,N185_F10
186,186,0.0,8.0,63.0,10,0,1,podium,N186_F10
187,187,8.0,8.0,63.0,10,1,1,podium,N187_F10
188,188,16.0,8.0,63.0,10,2,1,podium,N188_F10
189,189,24.0,8.0,63.0,10,3,1,podium,N189_F10


In [5]:
# Create lookup dictionary for quick node access
# Key: (floor, x_coord, y_coord) -> node_id
def create_node_lookup(df):
    """
    Create a dictionary for quick node lookup by (floor, x, y) coordinates.
    """
    lookup = {}
    for _, row in df.iterrows():
        key = (int(row['floor']), float(row['x']), float(row['y']))
        lookup[key] = int(row['node_id'])
    return lookup

node_lookup = create_node_lookup(position_df)

# Test lookup
print("Node lookup examples:")
print(f"  Floor 0, (0, 0): Node {node_lookup.get((0, 0.0, 0.0), 'N/A')}")
print(f"  Floor 0, (40, 16): Node {node_lookup.get((0, 40.0, 16.0), 'N/A')}")
print(f"  Floor 9, (0, 0): Node {node_lookup.get((9, 0.0, 0.0), 'N/A')} (last podium)")
print(f"  Floor 10, (8, 0): Node {node_lookup.get((10, 8.0, 0.0), 'N/A')} (first tower)")
print(f"  Floor 10, (0, 0): Node {node_lookup.get((10, 0.0, 0.0), 'N/A')} (should be N/A - edge removed)")
print(f"  Floor 10, (32, 16): Node {node_lookup.get((10, 32.0, 16.0), 'N/A')}")
print(f"  Floor 26, (16, 8): Node {node_lookup.get((26, 16.0, 8.0), 'N/A')}")

Node lookup examples:
  Floor 0, (0, 0): Node 0
  Floor 0, (40, 16): Node 17
  Floor 9, (0, 0): Node 162 (last podium)
  Floor 10, (8, 0): Node 181 (first tower)
  Floor 10, (0, 0): Node 180 (should be N/A - edge removed)
  Floor 10, (32, 16): Node 196
  Floor 26, (16, 8): Node N/A


---
## 4. Generate Connectivity Matrix

Structural elements:
- **Columns**: Vertical elements between floors
- **Beams X**: Horizontal elements along width
- **Beams Y**: Horizontal elements along depth  
- **Bracing XZ**: Diagonal bracing in front/back faces (X-Z plane)
- **Bracing YZ**: Diagonal bracing in side faces (Y-Z plane)

In [6]:
def generate_stepped_connectivity(position_df, node_lookup):
    """
    Generate connectivity matrix for stepped building with centered tower.
    
    Bracing strategy:
    - brace_xz: ALL bays on front/back faces (Y=0, Y=16), EVERY floor (X-pattern)
    - brace_yz: corner bays on side faces (edge X), EVERY floor (X-pattern)
    - brace_floor: X-pattern diagonal braces on floor plane (X-Y) for slab representation
    - brace_space: 3D space frame at podium-tower transition (setback areas)
    """
    connections = []
    element_id = 0
    
    # YZ brace bays (side faces): BOTH Y bays at edge X
    YZ_BRACE_BAYS = [(0.0, 8.0), (8.0, 16.0)]
    
    def get_coords(node_id):
        row = position_df[position_df['node_id'] == node_id].iloc[0]
        return row['x'], row['y'], row['z']
    
    def add_element(n1, n2, etype):
        nonlocal element_id
        x1, y1, z1 = get_coords(n1)
        x2, y2, z2 = get_coords(n2)
        length = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
        connections.append({
            'element_id': element_id,
            'node_i': n1,
            'node_j': n2,
            'element_type': etype,
            'length': round(length, 4)
        })
        element_id += 1
    
    def get_node(floor, x, y):
        return node_lookup.get((floor, float(x), float(y)))
    
    print("Generating structural elements...")
    
    for floor in range(TOTAL_FLOORS):
        if floor < PODIUM_FLOORS:
            x_coords = PODIUM_X_COORDS
        else:
            x_coords = TOWER_X_COORDS
        
        # 1. BEAMS X-DIRECTION
        for y in Y_COORDS:
            for i in range(len(x_coords) - 1):
                n1 = get_node(floor, x_coords[i], y)
                n2 = get_node(floor, x_coords[i + 1], y)
                if n1 is not None and n2 is not None:
                    add_element(n1, n2, 'beam_x')
        
        # 2. BEAMS Y-DIRECTION
        for x in x_coords:
            for i in range(len(Y_COORDS) - 1):
                n1 = get_node(floor, x, Y_COORDS[i])
                n2 = get_node(floor, x, Y_COORDS[i + 1])
                if n1 is not None and n2 is not None:
                    add_element(n1, n2, 'beam_y')
        
        # 3. COLUMNS
        if floor < TOTAL_FLOORS - 1:
            next_floor = floor + 1
            if next_floor < PODIUM_FLOORS:
                common_x = PODIUM_X_COORDS
            elif floor < PODIUM_FLOORS:
                common_x = TOWER_X_COORDS
            else:
                common_x = TOWER_X_COORDS
            
            for x in common_x:
                for y in Y_COORDS:
                    n1 = get_node(floor, x, y)
                    n2 = get_node(next_floor, x, y)
                    if n1 is not None and n2 is not None:
                        add_element(n1, n2, 'column')
        
        # 4. DIAGONAL BRACING XZ (front/back faces) - ALL BAYS, EVERY FLOOR, X-PATTERN
        if floor < TOTAL_FLOORS - 1:
            next_floor = floor + 1
            
            # Determine X coords for this floor-pair
            if next_floor < PODIUM_FLOORS:
                brace_x = PODIUM_X_COORDS
            else:
                brace_x = TOWER_X_COORDS
            
            for y in [Y_COORDS[0], Y_COORDS[-1]]:  # Front and back faces
                for i in range(len(brace_x) - 1):
                    x1, x2 = brace_x[i], brace_x[i + 1]
                    n_bl = get_node(floor, x1, y)
                    n_br = get_node(floor, x2, y)
                    n_tl = get_node(next_floor, x1, y)
                    n_tr = get_node(next_floor, x2, y)
                    
                    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                        # X-pattern: both diagonals
                        add_element(n_bl, n_tr, 'brace_xz')
                        add_element(n_br, n_tl, 'brace_xz')
        
        # 5. DIAGONAL BRACING YZ (side faces) - CORNER BAYS, EVERY FLOOR, X-PATTERN
        if floor < TOTAL_FLOORS - 1:
            next_floor = floor + 1
            
            if next_floor < PODIUM_FLOORS:
                side_x = [PODIUM_X_COORDS[0], PODIUM_X_COORDS[-1]]
            elif floor < PODIUM_FLOORS:
                side_x = [TOWER_X_COORDS[0], TOWER_X_COORDS[-1]]
            else:
                side_x = [TOWER_X_COORDS[0], TOWER_X_COORDS[-1]]
            
            for x in side_x:
                for (y1, y2) in YZ_BRACE_BAYS:
                    n_bf = get_node(floor, x, y1)
                    n_bb = get_node(floor, x, y2)
                    n_tf = get_node(next_floor, x, y1)
                    n_tb = get_node(next_floor, x, y2)
                    
                    if all(n is not None for n in [n_bf, n_bb, n_tf, n_tb]):
                        # X-pattern: both diagonals
                        add_element(n_bf, n_tb, 'brace_yz')
                        add_element(n_bb, n_tf, 'brace_yz')
        
        # 6. FLOOR BRACES (SLAB REPRESENTATION) - X-PATTERN (both diagonals per bay)
        for i in range(len(x_coords) - 1):
            for j in range(len(Y_COORDS) - 1):
                x1, x2 = x_coords[i], x_coords[i + 1]
                y1, y2 = Y_COORDS[j], Y_COORDS[j + 1]
                
                n_bl = get_node(floor, x1, y1)  # bottom-left
                n_br = get_node(floor, x2, y1)  # bottom-right
                n_tl = get_node(floor, x1, y2)  # top-left
                n_tr = get_node(floor, x2, y2)  # top-right
                
                if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                    # X-pattern: both diagonals
                    add_element(n_bl, n_tr, 'brace_floor')
                    add_element(n_br, n_tl, 'brace_floor')
    
    # 7. SPACE BRACES - 3D space frame at podium-tower transition (setback areas)
    transition_floor = PODIUM_FLOORS - 1  # Last podium floor (floor 12)
    tower_first_floor = PODIUM_FLOORS      # First tower floor (floor 13)
    
    # Left setback area (X: 0 to 8)
    for y in Y_COORDS:
        n_edge = get_node(transition_floor, 0.0, y)
        n_tower = get_node(transition_floor, TOWER_X_COORDS[0], y)
        if n_edge is not None and n_tower is not None:
            add_element(n_edge, n_tower, 'brace_space')
    
    for j in range(len(Y_COORDS) - 1):
        y1, y2 = Y_COORDS[j], Y_COORDS[j + 1]
        n1 = get_node(transition_floor, 0.0, y1)
        n2 = get_node(transition_floor, TOWER_X_COORDS[0], y1)
        n3 = get_node(transition_floor, 0.0, y2)
        n4 = get_node(transition_floor, TOWER_X_COORDS[0], y2)
        if all(n is not None for n in [n1, n2, n3, n4]):
            add_element(n1, n4, 'brace_space')
            add_element(n2, n3, 'brace_space')
    
    for y in Y_COORDS:
        n_podium_edge = get_node(transition_floor, 0.0, y)
        n_tower_up = get_node(tower_first_floor, TOWER_X_COORDS[0], y)
        if n_podium_edge is not None and n_tower_up is not None:
            add_element(n_podium_edge, n_tower_up, 'brace_space')
    
    for j in range(len(Y_COORDS) - 1):
        y1, y2 = Y_COORDS[j], Y_COORDS[j + 1]
        n_p1 = get_node(transition_floor, 0.0, y1)
        n_p2 = get_node(transition_floor, 0.0, y2)
        n_t1 = get_node(tower_first_floor, TOWER_X_COORDS[0], y1)
        n_t2 = get_node(tower_first_floor, TOWER_X_COORDS[0], y2)
        if all(n is not None for n in [n_p1, n_p2, n_t1, n_t2]):
            add_element(n_p1, n_t2, 'brace_space')
            add_element(n_p2, n_t1, 'brace_space')
    
    # Right setback area (X: 32 to 40)
    for y in Y_COORDS:
        n_edge = get_node(transition_floor, PODIUM_X_COORDS[-1], y)
        n_tower = get_node(transition_floor, TOWER_X_COORDS[-1], y)
        if n_edge is not None and n_tower is not None:
            add_element(n_edge, n_tower, 'brace_space')
    
    for j in range(len(Y_COORDS) - 1):
        y1, y2 = Y_COORDS[j], Y_COORDS[j + 1]
        n1 = get_node(transition_floor, TOWER_X_COORDS[-1], y1)
        n2 = get_node(transition_floor, PODIUM_X_COORDS[-1], y1)
        n3 = get_node(transition_floor, TOWER_X_COORDS[-1], y2)
        n4 = get_node(transition_floor, PODIUM_X_COORDS[-1], y2)
        if all(n is not None for n in [n1, n2, n3, n4]):
            add_element(n1, n4, 'brace_space')
            add_element(n2, n3, 'brace_space')
    
    for y in Y_COORDS:
        n_podium_edge = get_node(transition_floor, PODIUM_X_COORDS[-1], y)
        n_tower_up = get_node(tower_first_floor, TOWER_X_COORDS[-1], y)
        if n_podium_edge is not None and n_tower_up is not None:
            add_element(n_podium_edge, n_tower_up, 'brace_space')
    
    for j in range(len(Y_COORDS) - 1):
        y1, y2 = Y_COORDS[j], Y_COORDS[j + 1]
        n_p1 = get_node(transition_floor, PODIUM_X_COORDS[-1], y1)
        n_p2 = get_node(transition_floor, PODIUM_X_COORDS[-1], y2)
        n_t1 = get_node(tower_first_floor, TOWER_X_COORDS[-1], y1)
        n_t2 = get_node(tower_first_floor, TOWER_X_COORDS[-1], y2)
        if all(n is not None for n in [n_p1, n_p2, n_t1, n_t2]):
            add_element(n_p1, n_t2, 'brace_space')
            add_element(n_p2, n_t1, 'brace_space')
    
    print(f"Generated {element_id} elements")
    return pd.DataFrame(connections)

# Generate connectivity
connectivity_df = generate_stepped_connectivity(position_df, node_lookup)

print("\n" + "=" * 70)
print("CONNECTIVITY MATRIX")
print("=" * 70)
print(f"\nTotal elements: {len(connectivity_df)}")
print(f"\nElements by type:")
print(connectivity_df['element_type'].value_counts())

n_xz = len(connectivity_df[connectivity_df['element_type'] == 'brace_xz'])
n_yz = len(connectivity_df[connectivity_df['element_type'] == 'brace_yz'])
n_floor = len(connectivity_df[connectivity_df['element_type'] == 'brace_floor'])
n_space = len(connectivity_df[connectivity_df['element_type'] == 'brace_space'])
print(f"\nBrace ratio (XZ:YZ) = {n_xz}:{n_yz} = {n_xz/max(n_yz,1):.2f}:1")
print(f"Floor braces (slab): {n_floor}")
print(f"Space braces (3D): {n_space}")

Generating structural elements...
Generated 1984 elements

CONNECTIVITY MATRIX

Total elements: 1984

Elements by type:
element_type
brace_floor    416
brace_xz       396
column         372
beam_x         312
beam_y         260
brace_yz       200
brace_space     28
Name: count, dtype: int64

Brace ratio (XZ:YZ) = 396:200 = 1.98:1
Floor braces (slab): 416
Space braces (3D): 28


In [7]:
# Display sample connections
print("\n=== SAMPLE COLUMNS ===")
display(connectivity_df[connectivity_df['element_type'] == 'column'].head(10))

print("\n=== SAMPLE X-BEAMS ===")
display(connectivity_df[connectivity_df['element_type'] == 'beam_x'].head(10))

print("\n=== SAMPLE XZ-BRACING ===")
display(connectivity_df[connectivity_df['element_type'] == 'brace_xz'].head(10))


=== SAMPLE COLUMNS ===


,element_id,node_i,node_j,element_type,length
27,27,0,18,column,9.0
28,28,6,24,column,9.0
29,29,12,30,column,9.0
30,30,1,19,column,9.0
31,31,7,25,column,9.0
32,32,13,31,column,9.0
33,33,2,20,column,9.0
34,34,8,26,column,9.0
35,35,14,32,column,9.0
36,36,3,21,column,9.0



=== SAMPLE X-BEAMS ===


,element_id,node_i,node_j,element_type,length
0,0,0,1,beam_x,8.0
1,1,1,2,beam_x,8.0
2,2,2,3,beam_x,8.0
3,3,3,4,beam_x,8.0
4,4,4,5,beam_x,8.0
5,5,6,7,beam_x,8.0
6,6,7,8,beam_x,8.0
7,7,8,9,beam_x,8.0
8,8,9,10,beam_x,8.0
9,9,10,11,beam_x,8.0



=== SAMPLE XZ-BRACING ===


,element_id,node_i,node_j,element_type,length
45,45,0,19,brace_xz,12.0416
46,46,1,18,brace_xz,12.0416
47,47,1,20,brace_xz,12.0416
48,48,2,19,brace_xz,12.0416
49,49,2,21,brace_xz,12.0416
50,50,3,20,brace_xz,12.0416
51,51,3,22,brace_xz,12.0416
52,52,4,21,brace_xz,12.0416
53,53,4,23,brace_xz,12.0416
54,54,5,22,brace_xz,12.0416


In [8]:
# Create adjacency matrix
num_nodes = len(position_df)
adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

for _, row in connectivity_df.iterrows():
    i, j = int(row['node_i']), int(row['node_j'])
    adjacency_matrix[i, j] = 1
    adjacency_matrix[j, i] = 1  # Symmetric

print(f"Adjacency Matrix Shape: {adjacency_matrix.shape}")
print(f"Total connections (edges): {np.sum(adjacency_matrix) // 2}")

# Show a portion
print(f"\nAdjacency Matrix (first 18x18 - Floor 0 nodes):")
print(adjacency_matrix[:18, :18])

Adjacency Matrix Shape: (390, 390)
Total connections (edges): 1970

Adjacency Matrix (first 18x18 - Floor 0 nodes):
[[0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0]
 [1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0]
 [0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 0]
 [0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1]
 [0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0]]


---
## 5. Interactive 3D Visualization with Plotly

In [9]:
def create_3d_building_view(position_df, connectivity_df):
    """
    Create interactive 3D visualization of the stepped building.
    """
    fig = go.Figure()
    
    # Element colors and widths
    element_styles = {
        'column':       {'color': 'blue',      'width': 3},
        'beam_x':       {'color': 'green',     'width': 2},
        'beam_y':       {'color': 'orange',    'width': 2},
        'brace_xz':     {'color': 'red',       'width': 1.5},
        'brace_yz':     {'color': 'purple',    'width': 1.5},
        'brace_floor':  {'color': 'white',     'width': 1},
        'brace_space':  {'color': 'black',     'width': 2}
    }
    
    # Add elements by type
    for etype, style in element_styles.items():
        elements = connectivity_df[connectivity_df['element_type'] == etype]
        
        if len(elements) == 0:
            continue
        
        x_lines, y_lines, z_lines = [], [], []
        
        for _, row in elements.iterrows():
            n1 = position_df[position_df['node_id'] == row['node_i']].iloc[0]
            n2 = position_df[position_df['node_id'] == row['node_j']].iloc[0]
            
            x_lines.extend([n1['x'], n2['x'], None])
            y_lines.extend([n1['y'], n2['y'], None])
            z_lines.extend([n1['z'], n2['z'], None])
        
        fig.add_trace(go.Scatter3d(
            x=x_lines, y=y_lines, z=z_lines,
            mode='lines',
            name=etype.replace('_', ' ').title(),
            line=dict(color=style['color'], width=style['width']),
            hoverinfo='name'
        ))
    
    # Add nodes colored by floor
    fig.add_trace(go.Scatter3d(
        x=position_df['x'],
        y=position_df['y'],
        z=position_df['z'],
        mode='markers',
        name='Nodes',
        marker=dict(
            size=3,
            color=position_df['floor'],
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Floor', x=1.02)
        ),
        text=position_df['label'],
        hovertemplate='%{text}<br>X: %{x}m<br>Y: %{y}m<br>Z: %{z}m<extra></extra>'
    ))
    
    total_height = Z_COORDS[-1]
    podium_h = Z_COORDS[PODIUM_FLOORS - 1]
    
    fig.update_layout(
        title=dict(
            text=f'Stepped Building Structure - 3D View<br><sup>Podium (40x16m, {PODIUM_FLOORS} floors, {podium_h:.0f}m) + Centered Tower (24x16m, {TOTAL_FLOORS - PODIUM_FLOORS} floors) | Total Height: {total_height:.0f}m</sup>',
            x=0.5
        ),
        scene=dict(
            xaxis_title='X (Width) [m]',
            yaxis_title='Y (Depth) [m]',
            zaxis_title='Z (Height) [m]',
            aspectmode='data',
            camera=dict(eye=dict(x=1.5, y=1.5, z=0.7))
        ),
        width=1000,
        height=800,
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
    )
    
    return fig

# Create and show 3D view
fig_3d = create_3d_building_view(position_df, connectivity_df)
fig_3d.show()

---
## 6. Floor Plan Views

In [10]:
def create_floor_plan(position_df, connectivity_df, floor_num):
    """
    Create 2D floor plan view.
    """
    # Get nodes for this floor
    floor_nodes = position_df[position_df['floor'] == floor_num]
    floor_node_ids = set(floor_nodes['node_id'])
    
    # Get beams on this floor
    floor_beams = connectivity_df[
        (connectivity_df['node_i'].isin(floor_node_ids)) & 
        (connectivity_df['node_j'].isin(floor_node_ids)) &
        (connectivity_df['element_type'].isin(['beam_x', 'beam_y']))
    ]
    
    fig = go.Figure()
    
    # Draw beams
    for etype, color in [('beam_x', 'green'), ('beam_y', 'orange')]:
        beams = floor_beams[floor_beams['element_type'] == etype]
        for _, row in beams.iterrows():
            n1 = position_df[position_df['node_id'] == row['node_i']].iloc[0]
            n2 = position_df[position_df['node_id'] == row['node_j']].iloc[0]
            
            fig.add_trace(go.Scatter(
                x=[n1['x'], n2['x']],
                y=[n1['y'], n2['y']],
                mode='lines',
                line=dict(color=color, width=4),
                showlegend=False,
                hoverinfo='skip'
            ))
    
    # Draw column locations (nodes)
    fig.add_trace(go.Scatter(
        x=floor_nodes['x'],
        y=floor_nodes['y'],
        mode='markers+text',
        marker=dict(size=15, color='blue', symbol='square'),
        text=[f"({int(r['x'])},{int(r['y'])})" for _, r in floor_nodes.iterrows()],
        textposition='top center',
        name='Columns',
        hovertemplate='Node %{customdata}<br>X: %{x}m<br>Y: %{y}m<extra></extra>',
        customdata=floor_nodes['node_id']
    ))
    
    # Determine zone
    zone = 'Podium' if floor_num < PODIUM_FLOORS else 'Tower (Centered)'
    z_height = Z_COORDS[floor_num]
    
    fig.update_layout(
        title=f'Floor {floor_num} Plan View ({zone}) | Z = {z_height}m',
        xaxis_title='X [m]',
        yaxis_title='Y [m]',
        xaxis=dict(scaleanchor='y', scaleratio=1, range=[-5, 45]),
        yaxis=dict(range=[-5, 21]),
        width=900,
        height=500,
        showlegend=False
    )
    
    return fig

# Show podium floor (full footprint)
print("Floor 0 (Podium - Full Footprint 40m × 16m):")
fig_floor0 = create_floor_plan(position_df, connectivity_df, 0)
fig_floor0.show()

Floor 0 (Podium - Full Footprint 40m × 16m):


In [11]:
# Show tower floor (centered reduced footprint)
print("Floor 15 (Tower - Centered Footprint 24m × 16m):")
fig_floor15 = create_floor_plan(position_df, connectivity_df, 15)
fig_floor15.show()

Floor 15 (Tower - Centered Footprint 24m × 16m):


---
## 7. Front Elevation View (X-Z Plane)

In [12]:
def create_front_elevation(position_df, connectivity_df):
    """
    Create front elevation view (X-Z plane at Y=0).
    Shows the stepped profile of the building with centered tower.
    """
    # Get nodes at Y=0 (front face)
    front_nodes = position_df[position_df['y'] == 0]
    front_node_ids = set(front_nodes['node_id'])
    
    # Get elements involving these nodes
    front_elements = connectivity_df[
        (connectivity_df['node_i'].isin(front_node_ids)) & 
        (connectivity_df['node_j'].isin(front_node_ids))
    ]
    
    fig = go.Figure()
    
    # Element styles
    styles = {
        'column': ('blue', 3),
        'beam_x': ('green', 2),
        'brace_xz': ('red', 1.5)
    }
    
    # Draw elements
    for etype, (color, width) in styles.items():
        elements = front_elements[front_elements['element_type'] == etype]
        for _, row in elements.iterrows():
            n1 = position_df[position_df['node_id'] == row['node_i']].iloc[0]
            n2 = position_df[position_df['node_id'] == row['node_j']].iloc[0]
            
            fig.add_trace(go.Scatter(
                x=[n1['x'], n2['x']],
                y=[n1['z'], n2['z']],
                mode='lines',
                line=dict(color=color, width=width),
                name=etype.replace('_', ' ').title(),
                showlegend=False,
                hoverinfo='skip'
            ))
    
    # Draw nodes
    fig.add_trace(go.Scatter(
        x=front_nodes['x'],
        y=front_nodes['z'],
        mode='markers',
        marker=dict(size=4, color='black'),
        hovertemplate='Node %{customdata}<br>X: %{x}m<br>Z: %{y}m<extra></extra>',
        customdata=front_nodes['node_id'],
        showlegend=False
    ))
    
    # Add dimension annotations
    podium_top = Z_COORDS[PODIUM_FLOORS - 1]
    total_height = Z_COORDS[-1]
    podium_w = PODIUM_X_COORDS[-1]
    tower_w = TOWER_X_COORDS[-1] - TOWER_X_COORDS[0]
    
    fig.add_annotation(x=podium_w/2, y=-10, text=f"{podium_w:.0f}m (Podium)", showarrow=False, font=dict(size=12))
    fig.add_annotation(x=podium_w/2, y=total_height+10, text=f"{tower_w:.0f}m (Tower - Centered)", showarrow=False, font=dict(size=12))
    fig.add_annotation(x=-5, y=podium_top/2, text=f"{podium_top:.0f}m", showarrow=False, font=dict(size=10))
    fig.add_annotation(x=48, y=podium_top, text="Podium top", showarrow=False, font=dict(size=10))
    
    fig.update_layout(
        title=f'Front Elevation View (Y=0) | Podium: {podium_w:.0f}m, Tower: {tower_w:.0f}m | Height: {total_height:.0f}m',
        xaxis_title='X [m]',
        yaxis_title='Z [m]',
        xaxis=dict(range=[-15, 55]),
        yaxis=dict(range=[-20, total_height+25], scaleanchor='x', scaleratio=1),
        width=700,
        height=900,
        showlegend=False
    )
    
    return fig

fig_front = create_front_elevation(position_df, connectivity_df)
fig_front.show()

---
## 8. Side Elevation View (Y-Z Plane)

In [13]:
def create_side_elevation(position_df, connectivity_df):
    """
    Create side elevation view (Y-Z plane) showing the LEFT edge face.
    Podium: X=0 (left edge), Tower: X=8 (left edge).
    Both project onto the same Y-Z plane.
    """
    # Collect nodes on the left edge face
    podium_edge = position_df[(position_df['zone'] == 'podium') & (position_df['x'] == 0.0)]
    tower_edge = position_df[(position_df['zone'] == 'tower') & (position_df['x'] == 8.0)]
    side_nodes = pd.concat([podium_edge, tower_edge])
    side_node_ids = set(side_nodes['node_id'])
    
    # Get elements where BOTH nodes are on this edge
    side_elements = connectivity_df[
        (connectivity_df['node_i'].isin(side_node_ids)) & 
        (connectivity_df['node_j'].isin(side_node_ids))
    ]
    
    fig = go.Figure()
    
    styles = {
        'column': ('blue', 3),
        'beam_y': ('orange', 2),
        'brace_yz': ('red', 1.5),
    }
    
    for etype, (color, width) in styles.items():
        elements = side_elements[side_elements['element_type'] == etype]
        for _, row in elements.iterrows():
            n1 = position_df[position_df['node_id'] == row['node_i']].iloc[0]
            n2 = position_df[position_df['node_id'] == row['node_j']].iloc[0]
            
            fig.add_trace(go.Scatter(
                x=[n1['y'], n2['y']],
                y=[n1['z'], n2['z']],
                mode='lines',
                line=dict(color=color, width=width),
                showlegend=False,
                hoverinfo='skip'
            ))
    
    fig.add_trace(go.Scatter(
        x=side_nodes['y'],
        y=side_nodes['z'],
        mode='markers',
        marker=dict(size=4, color='black'),
        hovertemplate='Node %{customdata}<br>Y: %{x}m<br>Z: %{y}m<extra></extra>',
        customdata=side_nodes['node_id'],
        showlegend=False
    ))
    
    depth = Y_COORDS[-1]
    total_height = Z_COORDS[-1]
    podium_top = Z_COORDS[PODIUM_FLOORS - 1]
    
    fig.add_annotation(x=depth/2, y=-10, text=f"{depth:.0f}m", showarrow=False, font=dict(size=12))
    fig.add_annotation(x=20, y=podium_top, text="Podium top", showarrow=False, font=dict(size=10))
    
    fig.update_layout(
        title=f'Side Elevation View (Left Edge) | Depth: {depth:.0f}m | Height: {total_height:.0f}m',
        xaxis_title='Y [m]',
        yaxis_title='Z [m]',
        xaxis=dict(range=[-10, 30]),
        yaxis=dict(range=[-20, total_height+25], scaleanchor='x', scaleratio=1),
        width=450,
        height=900,
        showlegend=False
    )
    
    return fig

fig_side = create_side_elevation(position_df, connectivity_df)
fig_side.show()

---
## 9. Statistics Summary

In [14]:
print("=" * 70)
print("BUILDING STRUCTURAL SUMMARY")
print("=" * 70)

print("\n📐 GEOMETRY:")
print(f"   Podium:  {PODIUM_X_COORDS[-1]:.0f}m (W) × {Y_COORDS[-1]:.0f}m (D) × {Z_COORDS[PODIUM_FLOORS-1]:.0f}m (H)")
print(f"   Tower:   {TOWER_X_COORDS[-1] - TOWER_X_COORDS[0]:.0f}m (W) × {Y_COORDS[-1]:.0f}m (D) | Total Height: {Z_COORDS[-1]:.0f}m")
print(f"   Floors:  {TOTAL_FLOORS} total (Podium: 0-{PODIUM_FLOORS-1}, Tower: {PODIUM_FLOORS}-{TOTAL_FLOORS-1})")

print("\n🔢 NODES:")
print(f"   Total Nodes: {len(position_df)}")
print(f"   Podium nodes: {len(position_df[position_df['zone'] == 'podium'])} ({PODIUM_FLOORS} floors × {len(PODIUM_X_COORDS) * len(Y_COORDS)} nodes)")
print(f"   Tower nodes:  {len(position_df[position_df['zone'] == 'tower'])} ({TOTAL_FLOORS - PODIUM_FLOORS} floors × {len(TOWER_X_COORDS) * len(Y_COORDS)} nodes)")

print("\n🔗 ELEMENTS:")
for etype, count in connectivity_df['element_type'].value_counts().items():
    print(f"   {etype.replace('_', ' ').title():15s}: {count:5d}")
print(f"   {'TOTAL':15s}: {len(connectivity_df):5d}")

print("\n📏 ELEMENT LENGTHS:")
for etype in connectivity_df['element_type'].unique():
    lengths = connectivity_df[connectivity_df['element_type'] == etype]['length']
    print(f"   {etype.replace('_', ' ').title():15s}: min={lengths.min():.2f}m, max={lengths.max():.2f}m, avg={lengths.mean():.2f}m")

BUILDING STRUCTURAL SUMMARY

📐 GEOMETRY:
   Podium:  40m (W) × 16m (D) × 75m (H)
   Tower:   24m (W) × 16m (D) | Total Height: 153m
   Floors:  26 total (Podium: 0-12, Tower: 13-25)

🔢 NODES:
   Total Nodes: 390
   Podium nodes: 234 (13 floors × 18 nodes)
   Tower nodes:  156 (13 floors × 12 nodes)

🔗 ELEMENTS:
   Brace Floor    :   416
   Brace Xz       :   396
   Column         :   372
   Beam X         :   312
   Beam Y         :   260
   Brace Yz       :   200
   Brace Space    :    28
   TOTAL          :  1984

📏 ELEMENT LENGTHS:
   Beam X         : min=8.00m, max=8.00m, avg=8.00m
   Beam Y         : min=8.00m, max=8.00m, avg=8.00m
   Column         : min=6.00m, max=9.00m, avg=6.15m
   Brace Xz       : min=10.00m, max=12.04m, avg=10.10m
   Brace Yz       : min=10.00m, max=12.04m, avg=10.08m
   Brace Floor    : min=11.31m, max=11.31m, avg=11.31m
   Brace Space    : min=8.00m, max=12.81m, avg=10.75m


In [15]:
# Visualization of element statistics
fig_stats = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Element Count by Type', 'Element Length Distribution'],
    specs=[[{'type': 'pie'}, {'type': 'box'}]]
)

# Pie chart
counts = connectivity_df['element_type'].value_counts()
fig_stats.add_trace(
    go.Pie(
        labels=[e.replace('_', ' ').title() for e in counts.index],
        values=counts.values,
        hole=0.4
    ),
    row=1, col=1
)

# Box plot
for etype in connectivity_df['element_type'].unique():
    fig_stats.add_trace(
        go.Box(
            y=connectivity_df[connectivity_df['element_type'] == etype]['length'],
            name=etype.replace('_', ' ').title()
        ),
        row=1, col=2
    )

fig_stats.update_layout(title='Element Statistics', height=400, width=1000, showlegend=False)
fig_stats.update_yaxes(title_text='Length [m]', row=1, col=2)
fig_stats.show()

---
## 10. Export Data

In [16]:
# Export to CSV
position_df.to_csv('position_matrix.csv', index=False)
connectivity_df.to_csv('connectivity_matrix.csv', index=False)
np.savetxt('adjacency_matrix.csv', adjacency_matrix, delimiter=',', fmt='%d')

# Export to NumPy archive
np.savez('building_data.npz',
         adjacency_matrix=adjacency_matrix,
         podium_x_coords=PODIUM_X_COORDS,
         tower_x_coords=TOWER_X_COORDS,
         y_coords=Y_COORDS,
         z_coords=Z_COORDS)

print("Data exported:")
print("  - position_matrix.csv")
print("  - connectivity_matrix.csv")
print("  - adjacency_matrix.csv")
print("  - building_data.npz")

Data exported:
  - position_matrix.csv
  - connectivity_matrix.csv
  - adjacency_matrix.csv
  - building_data.npz


---
## Summary

This notebook models a **stepped building with centered setback**:

### Podium (Base)
- Dimensions: 40m × 16m (5 bays × 2 bays)
- Floors: 0-12 (13 floors)
- Height: 81m (9m ground + 11×6m typical)
- Grid: 6 × 3 = 18 column locations per floor
- X coords: [0, 8, 16, 24, 32, 40]

### Tower (Upper - Centered)
- Dimensions: 24m × 16m (3 bays × 2 bays)
- Floors: 13-25 (13 floors)
- Total Height: 153m
- Grid: 4 × 3 = 12 column locations per floor
- X coords: [8, 16, 24, 32] (edges removed, centered)

### Setback Pattern
- One column removed from **each side** (left X=0 and right X=40)
- Tower is **centered** on the podium

### Structural Elements
- **Columns**: Vertical members between floors
- **Beams**: Horizontal members (X and Y directions)
- **Bracing**: X-pattern diagonal bracing at corner bays on perimeter faces